In [ ]:
import ffmpeg
import os

input_file = "Y:\\Culture\\Videos\\Performers\\Eva de Vil\\Eva_de_Vil_Earn_It_With_CEI_and_Blue_Balls_Downloaded_2025_02_03_04_26_19.mp4"
output_dir = "H:\\Faces\\output\\frames"  # Specify your desired output directory
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

output_pattern = os.path.join(output_dir, 'frame_%04d.png')

(
    ffmpeg
    .input(input_file)
    .filter('fps', fps=1/2)
    .output(output_pattern)
    .run()
)

In [ ]:
import os
from mtcnn import MTCNN
import cv2
from PIL import Image
import numpy as np

# Initialize MTCNN
detector = MTCNN()

# Configure paths
frames_dir = "H:\\Faces\\output\\frames"  # Specify your desired output directory
faces_dir = "H:\\Faces\\output\\faces"    # Directory to save extracted faces
os.makedirs(faces_dir, exist_ok=True)

# Parameters
min_confidence = 0.95        # Minimum confidence for face detection
min_face_size = 100         # Minimum face size in pixels

# Process each frame
for frame_file in os.listdir(frames_dir):
    if not frame_file.endswith(('.png', '.jpg', '.jpeg')):
        continue
        
    # Read the image
    frame_path = os.path.join(frames_dir, frame_file)
    image = cv2.imread(frame_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Detect faces
    faces = detector.detect_faces(image_rgb)
    
    # Extract each face that meets our criteria
    for i, face in enumerate(faces):
        confidence = face['confidence']
        if confidence < min_confidence:
            continue
            
        # Get face coordinates
        x, y, width, height = face['box']
        
        # Add margin around face (20%)
        margin = 0.2
        x_margin = int(width * margin)
        y_margin = int(height * margin)
        
        # Calculate new coordinates with margin
        x1 = max(0, x - x_margin)
        y1 = max(0, y - y_margin)
        x2 = min(image.shape[1], x + width + x_margin)
        y2 = min(image.shape[0], y + height + y_margin)
        
        # Skip if face is too small
        if width < min_face_size or height < min_face_size:
            continue
        
        # Extract face region
        face_img = image[y1:y2, x1:x2]
        
        # Save the face
        face_filename = f"{os.path.splitext(frame_file)[0]}_face_{i}.jpg"
        face_path = os.path.join(faces_dir, face_filename)
        cv2.imwrite(face_path, face_img)
        
        print(f"Extracted face {i+1} from {frame_file} (confidence: {confidence:.2f})")

print("Face extraction complete!")

# Optional: Display some statistics
total_frames = len([f for f in os.listdir(frames_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
total_faces = len([f for f in os.listdir(faces_dir) if f.endswith('.jpg')])
print(f"\nProcessed {total_frames} frames")
print(f"Extracted {total_faces} faces")

In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [2]:
all_performers = stash_client.get_performers()
eva_de_vil = all_performers.filter(pl.col("stashapp_name").str.contains("Eva de Vil")).to_dicts()[0]

In [3]:
sample_scenes = stash_client.find_scenes_by_performers([eva_de_vil["stashapp_id"]])

In [ ]:
sample_scenes.to_dicts()[0]["stashapp_files"][0]["path"]

In [21]:
from libraries.FaceDatasetBuilder import FaceDatasetBuilder

# Example usage:
builder = FaceDatasetBuilder()

# Process a scene
scene_data = {
    'video_path': sample_scenes.to_dicts()[0]["stashapp_files"][0]["path"],
    'scene_id': sample_scenes.to_dicts()[0]["stashapp_stashdb_id"],
    'performer_ids': eva_de_vil["stashapp_stashdb_id"]
}
faces_extracted = builder.process_scene(
    scene_data['video_path'],
    scene_data['scene_id'],
    scene_data['performer_ids']
)

In [ ]:
# Later, verify faces
builder.verify_face('scene123_frame_0001_face_0', 'performer1')  # Verify
builder.verify_face('scene123_frame_0001_face_1', is_rejected=True)  # Reject